In [1]:
import pygame as pg
import random

%run settings.ipynb
vec = pg.math.Vector2

class Spritesheet:
    #load and parse spritesheets
    def __init__(self, filename):
        self.spritesheet = pg.image.load(filename).convert()
        
    def get_image(self, x, y, width, height):
        #grab an image from larger spritesheet
        image = pg.Surface((width, height))
        image.blit(self.spritesheet, (0,0), (x, y, width, height))
        image = pg.transform.scale(image, (int(width//1.40), int(height//1.40)))
        return image

class Player(pg.sprite.Sprite):
    def __init__(self, game):
        pg.sprite.Sprite.__init__(self)
        self.game = game
        #variable for animation
        self.walking  = False
        self.jumping  = False
        self.current_frame = 0
        self.last_update = 0
        self.load_images()
        self.image = self.standing_frames[0]
        self.rect = self.image.get_rect()
        self.rect.center = (WIDTH/2, HEIGHT/2)
        self.pos = vec(WIDTH/5, -HEIGHT*3/4)
        self.vel = vec(0, 0)
        self.acc = vec(0, 0)
    
    def load_images(self):
        self.standing_frames = [self.game.spritesheet.get_image(height=201, width=120, y=406, x=690), 
                                               self.game.spritesheet.get_image(height=191,width=120,y=1063, x=614)]
        #remove background
        for frame in self.standing_frames:
            frame.set_colorkey(BLACK)

        self.walk_frames_r = [self.game.spritesheet.get_image(height=201, width=120, y=860, x=678),
                                             self.game.spritesheet.get_image(height=207,width=120,y=1458, x=692)]
        #remove background
        for frame in self.walk_frames_r:
            frame.set_colorkey(BLACK)
            
        self.walk_frames_l = []
        for frame in self.walk_frames_r:
            #1-horizontal 2-vertical
            self.walk_frames_l.append(pg.transform.flip(frame, True, False))
 
        #remove background
        for frame in self.walk_frames_l:
            frame.set_colorkey(BLACK)
            
        self.jump_frame = [self.game.spritesheet.get_image(height=181, width=150, y=763, x=382)]
                              
                  
    def jump(self):
        self.rect.x += 1
        hit_p = pg.sprite.spritecollide(self, self.game.platforms, False)
        hit_g = pg.sprite.spritecollide(self, self.game.grounds, False)
        self.rect.x -= 1       
        if hit_p or hit_g:
            self.vel.y = PLAYER_JUMP
            
    def shoot(self):
        if self.game.balls == 0:
            pass
        else:
            bullet = Bullet(self.game, self.rect.centerx, self.rect.top)
            self.game.bullets.add(bullet)
            self.game.all_sprites.add(bullet)
            self.game.balls -= 1
            
    def update(self):
        self.animate()
        
        #x and y accelaration vector
        self.acc = vec(0, PLAYER_GRAV)
        self.acc.x = PLAYER_ACC
        
        keys = pg.key.get_pressed()
        if keys[pg.K_LEFT]:
            self.acc.x += -PLAYER_ACC*2
            
#         if keys[pg.K_RIGHT]:
#         self.acc.x = PLAYER_ACC

        #The faster, the higher the friction
        self.acc.x += self.vel.x * PLAYER_FRICTION
        self.vel += self.acc
        #motion equation: vel * 1/2 acc
        self.pos += self.vel + 0.5 * self.acc
        
        #screen's size y
        if self.pos.y >= HEIGHT:
            self.pos.y = HEIGHT
        #screen's size x
        if self.pos.x > WIDTH:
            self.pos.x = WIDTH
        if self.pos.x < 0:
            self.pos.x  = 0
        self.rect.midbottom = self.pos
    
    def animate(self):
        now = pg.time.get_ticks()
        
        if self.vel.x != 0:
            self.walking = True
        else:
            self.walking = False
        #walk animation
        if self.walking:
            if now - self.last_update > 200:
                self.last_update = now
                self.current_frame = (self.current_frame + 1) % len(self.walk_frames_r)
                bottom = self.rect.bottom
                if self.vel.x > 0:
                    self.image = self.walk_frames_r[self.current_frame]
                else:
                    self.image = self.walk_frames_l[self.current_frame]
                self.rect = self.image.get_rect()
                self.rect.bottom = bottom
            
        if not self.jumping and not self.walking:
            if now - self.last_update > 350:
                self.last_update = now
                self.current_frame = (self.current_frame + 1) % len(self.standing_frames)
                bottom = self.rect.bottom
                self.image = self.standing_frames[self.current_frame]
                self.rect = self.image.get_rect()
                self.rect.bottom = bottom
                
class Platform(pg.sprite.Sprite):
    #position and dimension of the platform
    def __init__(self, game,x, y, w, h):
        pg.sprite.Sprite.__init__(self)
#         self.image = pg.Surface((w, h))
#         self.image.fill(GREEN)
        self.game = game
        self.image = self.game.spritesheet.get_image(height=20,width=250, y=960, x=0)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

class Ground(pg.sprite.Sprite):
    #position and dimension of the platform
    def __init__(self, x, y, w, h):
        pg.sprite.Sprite.__init__(self)
        self.image = pg.Surface((w, h))
        self.image.fill(GREEN)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

class Bullet(pg.sprite.Sprite):
    def __init__(self, game, x, y):
        pg.sprite.Sprite.__init__(self)
#         self.image = pg.Surface((50,50))
#         self.image.fill(BLUE)
        self.game=game
        self.image = self.game.spritesheet.get_image(height=49,width=51,y=1168, x=852)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.bottom = y
        self.rect.centerx = x
        self.speedx = 10
        
    def update(self):
        self.rect.x += self.speedx
        if self.rect.right < 0:
            self.kill()
            
            
class Ammo(pg.sprite.Sprite):
    def __init__(self, game):
        self.groups = game.all_sprites, game.ammos
        pg.sprite.Sprite.__init__(self, self.groups)
        self.game = game
        #set mob image
        self.image = self.game.spritesheet.get_image(height=49,width=51,y=1168, x=852)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.centerx = random.choice([-100, WIDTH+100])
        self.vx = random.randrange(5,8)
        if self.rect.centerx > WIDTH:
            self.vx *= -1
        self.rect.y = random.choice(COINS_LIST)
        self.vy = 0
        self.dy = 0.5
        
    def update(self):
        self.rect.x += self.vx
        self.rect.y += self.vy
        if self.rect.centerx < 0:
            self.kill()    
            
class Mob(pg.sprite.Sprite):
    def __init__(self, game, image):
        self.groups = game.all_sprites, game.mobs
        pg.sprite.Sprite.__init__(self, self.groups)
        self.game = game
#         self.image = pg.Surface(image)
#         self.image.fill(RED)
        #set mob image
        self.image = self.game.spritesheet.get_image(height=145, width=145, y=763, x=534)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.centerx = random.choice([-100, WIDTH+100])
        self.vx = random.randrange(4,8)
        if self.rect.centerx > WIDTH:
            self.vx *= -1
        self.rect.y = random.choice([150, HEIGHT*3/4+20])
        self.vy = 0
        self.dy = 0.1
        
    def update(self):
        self.rect.x += self.vx
        self.vy += self.dy
        if self.vy > 3 or self.vy < -3:
            self.dy *= -1
        center = self.rect.center
        self.rect.y += self.vy
        if self.rect.centerx < 0:
            self.game.score += 100
            self.kill()
###################################################
class Runner(pg.sprite.Sprite):
    def __init__(self, game):
        self.groups = game.all_sprites, game.runners
        pg.sprite.Sprite.__init__(self, self.groups)
        self.game = game
        
        #set RUNNER image
        self.image = self.game.spritesheet.get_image(height=201, width=120, y=860, x=678)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.centerx = WIDTH
        self.vx = random.randrange(4,8)
        if self.rect.centerx > WIDTH:
            self.vx *= -1
        self.rect.y = HEIGHT*3/4+20
        
    def update(self):
        self.rect.x -= self.vx
        if self.rect.centerx < 0:
            self.kill()

            
class Powerups(pg.sprite.Sprite):
    def __init__(self, game):
        self.groups = game.all_sprites, game.powerups
        pg.sprite.Sprite.__init__(self, self.groups)
        self.game = game
        #set mob image
        self.image = self.game.spritesheet.get_image(height=80,width=41,y=1733,x=900)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.centerx = random.choice([-100, WIDTH+100])
        self.vx = random.randrange(2,4)
        if self.rect.centerx > WIDTH:
            self.vx *= -1
        self.rect.y = random.choice(COINS_LIST)
        self.vy = 0
        self.dy = 0.5
        
    def update(self):
        self.rect.x += self.vx
        self.rect.y += self.vy
        if self.rect.centerx < 0:
            self.kill()    
            
            
class Coins(pg.sprite.Sprite):
    def __init__(self, game):
        self.groups = game.all_sprites, game.coins
        pg.sprite.Sprite.__init__(self, self.groups)
        self.game = game
#         self.image = pg.Surface(image)
#         self.image.fill(RED)
        #set mob image
        self.image = self.game.spritesheet.get_image(height=84, width=84, y=1931, x=698)
        self.image.set_colorkey(BLACK)
        self.rect = self.image.get_rect()
        self.rect.centerx = random.choice([-100, WIDTH+100])
        self.vx = random.randrange(4,8)
        if self.rect.centerx > WIDTH:
            self.vx *= -1
        self.rect.y = random.choice(COINS_LIST)
        self.vy = 0
        self.dy = 0.5
        
    def update(self):
        self.rect.x += self.vx
        self.vy += self.dy
        if self.vy > 3 or self.vy < -3:
            self.dy *= -1
        center = self.rect.center
        self.rect.y += self.vy
        if self.rect.centerx < 0:
            self.kill()



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [22]:
import os



AttributeError: module 'os' has no attribute 'dir'